In [1]:
# Method 1

import os
import json
from collections import defaultdict

# List of folders
folders = ['01', '02']
common_directory = '03'

# Create the common directory if it doesn't exist
if not os.path.exists(common_directory):
    os.makedirs(common_directory)

# Dictionary to hold name occurrences
name_occurrences = defaultdict(int)

# Move images to common directory and rename if necessary
for folder in folders:
    for file_name in os.listdir(folder):
        if file_name.endswith('.png'):
            name_occurrences[file_name] += 1
            suffix = ''
            if name_occurrences[file_name] > 1:
                suffix = '_' + chr(96 + name_occurrences[file_name])
                new_file_name = file_name[:-4] + suffix + '.png'
            else:
                new_file_name = file_name
            os.rename(os.path.join(folder, file_name), os.path.join(common_directory, new_file_name))

# Initialize merged coco data
merged_coco_data = {
    "images": [],
    "annotations": [],
    "categories": [{"id": 1, "name": "lefthand"}, {"id": 2, "name": "righthand"}]
}

# Merge json files
last_image_id = 0
last_annotation_id = 0
for folder in folders:
    json_file = [f for f in os.listdir(folder) if f.endswith('.json')][0]
    with open(os.path.join(folder, json_file)) as file:
        coco_data = json.load(file)

        # Update image ids and filenames
        for image in coco_data['images']:
            last_image_id += 1
            image['id'] = last_image_id
            occurrences = name_occurrences[image['file_name']]
            if occurrences > 1:
                suffix = '_' + chr(96 + occurrences)
                image['file_name'] = image['file_name'][:-4] + suffix + '.png'
            merged_coco_data['images'].append(image)

        # Update annotation ids and round segmentation values
        for annotation in coco_data['annotations']:
            last_annotation_id += 1
            annotation['id'] = last_annotation_id
            annotation['image_id'] += last_image_id - len(coco_data['images'])
            annotation['segmentation'] = [[round(value * 2) / 2 for value in seg] for seg in annotation['segmentation']]
            merged_coco_data['annotations'].append(annotation)

# Write merged json to file
with open('merged_coco.json', 'w') as file:
    json.dump(merged_coco_data, file)

In [4]:
import os
import shutil
import json
import glob

# Step 1: Copy and Rename Images
common_directory = '03'
os.makedirs(common_directory, exist_ok=True)

folders = ['01', '02']

for folder in folders:
    images_path = os.path.join(folder, '*.png')
    images = glob.glob(images_path)
    for image_path in images:
        basename = os.path.basename(image_path)
        new_path = os.path.join(common_directory, basename)
        
        # Rename if file with the same name exists
        suffix = ord('a') - 1
        while os.path.exists(new_path):
            suffix += 1
            new_name = os.path.splitext(basename)[0] + "_" + chr(suffix) + ".png"
            new_path = os.path.join(common_directory, new_name)
        
        shutil.copy(image_path, new_path)

# Step 2: Merge JSON files
merged_data = {
    'images': [],
    'annotations': [],
    'categories': []
}

image_id_offset = 0
annotation_id_offset = 0

for folder in folders:
    json_file = [f for f in os.listdir(folder) if f.endswith('.json')][0]
    with open(os.path.join(folder, json_file)) as file:
        data = json.load(file)
        
        # Update image file_name and ids
        for image in data['images']:
            basename = image['file_name']
            suffix = ord('a') - 1
            while os.path.exists(os.path.join(common_directory, basename)):
                suffix += 1
                basename = os.path.splitext(image['file_name'])[0] + "_" + chr(suffix) + ".png"

            image['file_name'] = basename
            image['id'] += image_id_offset
            merged_data['images'].append(image)
        
        # Update annotations ids and round segmentation values
        for annotation in data['annotations']:
            annotation['id'] += annotation_id_offset
            annotation['image_id'] += image_id_offset
            annotation['segmentation'] = [[round(x * 2) / 2 for x in seg] for seg in annotation['segmentation']]
            merged_data['annotations'].append(annotation)

        image_id_offset = max(image['id'] for image in data['images']) + 1
        annotation_id_offset = max(annotation['id'] for annotation in data['annotations']) + 1
        
    # Copy categories (assuming they are the same across folders)
    if not merged_data['categories']:
        merged_data['categories'] = data['categories']

# Write merged data to file
with open('merged_annotations.json', 'w') as file:
    json.dump(merged_data, file)

In [15]:
# Best Version Method 3
import os
import shutil
import json
import math

def round_to_nearest_half(number):
    return round(number * 2) / 2

def merge_datasets(folders, common_directory, t):
    os.makedirs(common_directory, exist_ok=True)
    image_id_offset = 0
    annotation_id_offset = 0

    final_json = {
        "images": [],
        "annotations": [],
        "categories": [{"id": 1, "name": "leftthumb"}, {"id": 2, "name": "leftindex"}, 
                       {"id": 3, "name": "leftfour"}, {"id": 4, "name": "rightthumb"},
                       {"id": 5, "name": "rightindex"}, {"id": 6, "name": "rightfour"},
                       {"id": 7, "name": "emptyscanner"}]
    }

    name_counter = {}

    for folder in folders:
        print('opening..', folder)
        with open(os.path.join(folder, f'{t}.json'), 'r') as json_file:
            annotations_data = json.load(json_file)

        for i, image in enumerate(annotations_data["images"]):
            file_name = image["file_name"]
            src_image_path = os.path.join(folder, file_name)
            print(i, 'processing for', file_name)
            
            # Renaming logic if filename exists
            base_name, ext = os.path.splitext(file_name)
            suffix = name_counter.get(base_name, 0)
            if suffix:
                new_file_name = f"{base_name}_{chr(ord('a') + suffix)}{ext}"
            else:
                new_file_name = file_name
            
            name_counter[base_name] = suffix + 1

            # Updating the file_name in the JSON and copy to common directory
            image["file_name"] = new_file_name
            dest_image_path = os.path.join(common_directory, new_file_name)
            shutil.copyfile(src_image_path, dest_image_path)
            image["id"] += image_id_offset
            final_json["images"].append(image)

        for annotation in annotations_data["annotations"]:
            # Round the segmentation values
            annotation["segmentation"] = [[round_to_nearest_half(value) for value in seg] for seg in annotation["segmentation"]]
            annotation["bbox"] = [round_to_nearest_half(value) for value in annotation["bbox"]]
            annotation["image_id"] += image_id_offset
            annotation["id"] += annotation_id_offset
            final_json["annotations"].append(annotation)

        # Updating the image and annotation id offsets
        image_id_offset += len(annotations_data["images"])
        annotation_id_offset += len(annotations_data["annotations"])

    # Write the merged JSON to the common directory
    with open(os.path.join(common_directory, f'{t}.json'), 'w') as merged_file:
        json.dump(final_json, merged_file)

# List of folders to be merged
t = 'train'
folders = ['21Aug23_DDA', '21Aug23_DDA_B', '22Aug23_DDA', '22Aug23_DDA_b', '22Aug23_DDA_b']
# Common directory for merged data|
common_directory = f'final_23aug_{t}_processed_v2'
merge_datasets(folders, common_directory, t)

opening.. 21Aug23_DDA
0 processing for 21613.png
1 processing for 21773.png
2 processing for 21823.png
3 processing for 21967.png
4 processing for 22006.png
5 processing for 22008.png
6 processing for 22151.png
7 processing for 22193.png
8 processing for 22368.png
9 processing for 22394.png
10 processing for 22464.png
11 processing for 22488.png
12 processing for 22515.png
13 processing for 22536.png
14 processing for 22547.png
15 processing for 22660.png
16 processing for 22966.png
17 processing for 23029.png
18 processing for 23052.png
19 processing for 23097.png
20 processing for 23121.png
21 processing for 23195.png
22 processing for 23197.png
23 processing for 23216.png
24 processing for 23256.png
25 processing for 23270.png
26 processing for 23284.png
27 processing for 23298.png
28 processing for 23342.png
29 processing for 23435.png
30 processing for 23468.png
31 processing for 23626.png
32 processing for 23655.png
33 processing for 23739.png
34 processing for 23763.png
35 proce

In [11]:
'''import os
import shutil
import json
import math

def round_to_nearest_half(number):
    return round(number * 2) / 2

def merge_datasets(folders, common_directory):
    os.makedirs(common_directory, exist_ok=True)
    image_id_offset = 0
    annotation_id_offset = 0

    final_json = {
        "images": [],
        "annotations": [],
        "categories": [{"id": 1, "name": "lefthand"}, {"id": 2, "name": "righthand"}]
    }

    name_counter = {}

    for folder in folders:
        with open(os.path.join(folder, 'annotations.json'), 'r') as json_file:
            annotations_data = json.load(json_file)

        for image in annotations_data["images"]:
            file_name = image["file_name"]
            src_image_path = os.path.join(folder, file_name)
            
            # Renaming logic if filename exists
            base_name, ext = os.path.splitext(file_name)
            suffix = name_counter.get(base_name, 0)
            if suffix:
                new_file_name = f"{base_name}_{chr(ord('a') + suffix)}{ext}"
            else:
                new_file_name = file_name
            
            name_counter[base_name] = suffix + 1

            # Updating the file_name in the JSON and copy to common directory
            image["file_name"] = new_file_name
            dest_image_path = os.path.join(common_directory, new_file_name)
            shutil.copyfile(src_image_path, dest_image_path)
            image["id"] += image_id_offset
            final_json["images"].append(image)

        for annotation in annotations_data["annotations"]:
            # Round the segmentation and bbox values
            annotation["segmentation"] = [[round_to_nearest_half(value) for value in seg] for seg in annotation["segmentation"]]
            annotation["bbox"] = [round_to_nearest_half(value) for value in annotation["bbox"]]
            annotation["image_id"] += image_id_offset
            annotation["id"] += annotation_id_offset
            final_json["annotations"].append(annotation)

        # Updating the image and annotation id offsets
        image_id_offset += len(annotations_data["images"])
        annotation_id_offset += len(annotations_data["annotations"])

    # Write the merged JSON to the common directory
    with open(os.path.join(common_directory, 'merged_annotations.json'), 'w') as merged_file:
        json.dump(final_json, merged_file, indent=4)

# List of folders to be merged
folders = ['01', '02']
# Common directory for merged data
common_directory = '03'
merge_datasets(folders, common_directory)'''